In [1]:
import pandas as pd
import re

In [ ]:
df = pd.read_json('1414384\MathStackExchangeAPI_Part_1_TimeStamps_1512760268_1535031491.json\MathStackExchangeAPI_Part_1_TimeStamps_1512760268_1535031491.json')[['answers','tags']]
df.info()

In [3]:
df_2 = pd.read_json("1414384\MathStackExchangeAPI_Part_2_TimeStamps_1484447260_1512760268.json\MathStackExchangeAPI_Part_2_TimeStamps_1484447260_1512760268.json")[['answers','tags']]
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163100 entries, 0 to 163099
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   answers  128942 non-null  object
 1   tags     163100 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [5]:
# сверху первые два джейсона
# а тут будут остальные

df_3 = pd.read_json("1414384\MathStackExchangeAPI_Part_3_TimeStamps_1470978327_1484447260.json\MathStackExchangeAPI_Part_3_TimeStamps_1470978327_1484447260.json")[['answers','tags']]
df_4 = pd.read_json("1414384\MathStackExchangeAPI_Part_4_TimeStamps_1439982779_1470978327.json\MathStackExchangeAPI_Part_4_TimeStamps_1439982779_1470978327.json")[['answers','tags']]
df_5 = pd.read_json("1414384\MathStackExchangeAPI_Part_5_TimeStamps_1416592625_1439982779.json\MathStackExchangeAPI_Part_5_TimeStamps_1416592625_1439982779.json")[['answers','tags']]
df_6 = pd.read_json("1414384\MathStackExchangeAPI_Part_6_TimeStamps_1393683354_1416592625.json\MathStackExchangeAPI_Part_6_TimeStamps_1393683354_1416592625.json")[['answers','tags']]
df_7 = pd.read_json("1414384\MathStackExchangeAPI_Part_7_TimeStamps_1367752303_1393683354.json\MathStackExchangeAPI_Part_7_TimeStamps_1367752303_1393683354.json")[['answers','tags']]
df_8 = pd.read_json("1414384\MathStackExchangeAPI_Part_8_TimeStamps_1269700400_1367752303.json\MathStackExchangeAPI_Part_8_TimeStamps_1269700400_1367752303.json")[['answers','tags']]


In [6]:
df = pd.concat([df_3, df_4, df_5, df_6, df_7, df_8])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697538 entries, 0 to 126637
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   answers  605757 non-null  object
 1   tags     697538 non-null  object
dtypes: object(2)
memory usage: 16.0+ MB


In [8]:
df = df[~df['answers'].isnull()]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 605757 entries, 0 to 126637
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   answers  605757 non-null  object
 1   tags     605757 non-null  object
dtypes: object(2)
memory usage: 13.9+ MB


In [10]:
df=df.reset_index(drop=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605757 entries, 0 to 605756
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   answers  605757 non-null  object
 1   tags     605757 non-null  object
dtypes: object(2)
memory usage: 9.2+ MB


In [12]:
df.head()

,answers,tags
0,"[{'answer_id': 2098361, 'body_markdown': 'For ...","[inequality, floor-function]"
1,"[{'answer_id': 2098260, 'body_markdown': 'It s...","[real-analysis, measure-theory]"
2,"[{'answer_id': 2098256, 'body_markdown': 'No. ...","[real-analysis, integration, functional-analysis]"
3,"[{'answer_id': 2098281, 'body_markdown': 'Spec...",[graphing-functions]
4,"[{'answer_id': 2098252, 'body_markdown': 'The ...","[complex-analysis, complex-numbers]"


In [16]:
from html import unescape
from tqdm import tqdm

In [18]:
# собираем это в один 'плоский' датафрейм
# и заодно избавляем от html чтобы два раза не ходить
rows = []
i=list(df['tags'].keys())[0]
#progress = 0
text_num = 0

for answers in tqdm(df['answers']):
    if answers is not None:
        for answer in answers:
            row = {}
            markdown = answer['body_markdown']
            row['text_id']=text_num
            row['markdown']= unescape(markdown) # собствено преобразование html-я
            row['tags']=df['tags'][i]
            rows.append(row)
            text_num +=1
    i+=1 
df_rows = pd.DataFrame.from_dict(rows)

100%|███████████████████████████████████████████████████████████████████████| 605757/605757 [01:00<00:00, 10087.55it/s]


In [21]:
def extract_substrings(text): 
    # находим фразы, начинающиеся и заканчивающиеся знаками доллара или начинающиеся и заканчивающиеся \\begin{} и \\end{} 
    matches = re.findall(r'(?<=.)?(\$\$.+?\$\$|\$.+?\$|\\begin\{(.*?)\}(.*?)\\end\{\2\})(?=.*$)', text, flags=re.DOTALL) 
    return [match[0] if match[0] else f'/begin{{{match[1]}}}{match[2]}/end{{{match[1]}}}' for match in matches] 
    # разделить поэтапно

def process_json_file(inp_df):
    data = {'body_markdown': inp_df['markdown']}
    
    # преобразуем данные в DataFrame, добавляя индекс
    df = pd.DataFrame(data)
    print('began extraction')
    # воздаем новый столбец для хранения вычлененных подстрок
    df['formulae'] = df['body_markdown'].apply(extract_substrings)
    df['tags'] = inp_df['tags']
    df['text_id'] = inp_df['text_id']
    print("applied extraction")
    
    # создаем новый DataFrame для хранения вычлененных подстрок
    extr_frms=[]
    for text_id, sublist in tqdm(enumerate(df['formulae'])):
        for ind, item in enumerate(sublist):
            entry = {}
            entry['text_id'] = text_id
            entry['formula_index'] = ind
            entry['original_formula'] = item
            extr_frms.append(entry)
            
    extracted_phrases_df = pd.DataFrame.from_dict(extr_frms)
    print("created individual formulae dataframe")
    
    # заменяем подстроки на [[i]] в исходной таблице
    for index, row in tqdm(df.iterrows()):
        for i, formula in enumerate(row['formulae']):
            df.at[index, 'body_markdown'] = df.at[index, 'body_markdown'].replace(formula, f'[[{i}]]', 1)
    print("changed all formulae in text to [[{formula_num}]]")
    # возвращаем результат
    return df, extracted_phrases_df

res_df, extracted_phrases_dataframe = process_json_file(df_rows)


began extraction
applied extraction


1027017it [00:12, 80599.35it/s]


created individual formulae dataframe


1027017it [29:05, 588.40it/s] 

changed all formulae in text to [[{formula_num}]]


In [22]:
extracted_phrases_dataframe.head(10)

,text_id,formula_index,original_formula
0,0,0,$0\lt a\lt 1$
1,0,1,$b\gt 0$
2,0,2,"$$1-\frac{\lfloor a\cdot\min\{N,K\}\rfloor}{N}..."
3,0,3,"$$-\frac{(\lfloor a\cdot\mathrm{min}\{N,K\}\rf..."
4,0,4,$K\ge N$
5,0,5,$$\begin{align}(1)&\iff -\frac{(\lfloor a\cdot...
6,0,6,$(2)$
7,0,7,$(N-\lfloor aN\rfloor)(N-aN)\gt 0$
8,0,8,$$\begin{align}(2)&\iff (N-aN)\lfloor aN\rfloo...
9,0,9,$0\lt a\lt 1$


In [23]:
res_df.head(10)

,body_markdown,formulae,tags,text_id
0,For [[0]] and [[1]] such that \r\n[[2]]\r\nthe...,"[$0\lt a\lt 1$, $b\gt 0$, $$1-\frac{\lfloor a\...","[inequality, floor-function]",0
1,It suffices to look at the case [[0]]. Let [[1...,"[$n=2$, $E$, $E_1 = \{x\in E : f_1(x) \ge f_2(...","[real-analysis, measure-theory]",1
2,"Since [[0]] are measurable, so are [[1]], and ...","[$f_i$, $f_i - f_j$, $L_{ij} = \{x: f_i(x) \le...","[real-analysis, measure-theory]",2
3,No. Take [[0]] to be the indicator of [[1]]. T...,"[$g$, $[0,1]$, $\|g\|_2 = 1$, $$\int_{\Bbb R} ...","[real-analysis, integration, functional-analysis]",3
4,Specify that you want a parametric plot and ex...,[],[graphing-functions],4
5,The absolute value is very simple:\r\n\r\n[[0]...,"[$$|z^4|=|-1+\sqrt3 i|=2$$, $$|z|=\sqrt[4]2$$]","[complex-analysis, complex-numbers]",5
6,Observe that [[0]]. Then the roots of the equa...,"[$-1+\sqrt{3}i=2e^{\frac{2\pi i}{3}}$, $$\r\nz...","[complex-analysis, complex-numbers]",6
7,HINT:\r\n\r\ncis[[0]]\r\n\r\n[[1]]cis[[2]]\r\n...,[$(2y)-1=\cos2y+i\sin2y-1=-2\sin^2y+2i\sin y\c...,"[calculus, algebra-precalculus, trigonometry]",7
8,"If [[0]], it's known that for [[1]] a prime, [...","[$a\in F$, $p$, $X^p-a$, $F$, $X^p-a$, $F$, $X...","[abstract-algebra, polynomials, irreducible-po...",8
9,"Sure, you can do that -- many math textbooks h...",[],"[proof-writing, article-writing]",9


In [24]:
final = [(t_id, text, form, tags) for (t_id, text, form, tags) in zip(res_df['text_id'],
                                                          res_df['body_markdown'],
                                                        res_df['formulae'],
                                                        res_df['tags']) if len(form)]

In [25]:
len(final)

926327

In [26]:
fin_df = pd.DataFrame.from_records(final, columns =['text_id','body_markdown', 'formulae', 'tags'])

In [27]:
fin_df


,text_id,body_markdown,formulae,tags
0,0,For [[0]] and [[1]] such that \r\n[[2]]\r\nthe...,"[$0\lt a\lt 1$, $b\gt 0$, $$1-\frac{\lfloor a\...","[inequality, floor-function]"
1,1,It suffices to look at the case [[0]]. Let [[1...,"[$n=2$, $E$, $E_1 = \{x\in E : f_1(x) \ge f_2(...","[real-analysis, measure-theory]"
2,2,"Since [[0]] are measurable, so are [[1]], and ...","[$f_i$, $f_i - f_j$, $L_{ij} = \{x: f_i(x) \le...","[real-analysis, measure-theory]"
3,3,No. Take [[0]] to be the indicator of [[1]]. T...,"[$g$, $[0,1]$, $\|g\|_2 = 1$, $$\int_{\Bbb R} ...","[real-analysis, integration, functional-analysis]"
4,5,The absolute value is very simple:\r\n\r\n[[0]...,"[$$|z^4|=|-1+\sqrt3 i|=2$$, $$|z|=\sqrt[4]2$$]","[complex-analysis, complex-numbers]"
...,...,...,...,...
926322,1026986,Another one that is understandable by high sch...,"[$n$, $n^2$, $n$, $(2k)^2 = 2(2k^2)$, $(2k+1)^...","[elementary-number-theory, proof-writing, radi..."
926323,1026987,Let [[0]] be the polynomial equations this hav...,"[$x^2-2=0$, $\pm1,\pm2$, $\pm \sqrt2$]","[elementary-number-theory, proof-writing, radi..."
926324,1026989,This proof is by descent (aka the Well-orderin...,"[$n$, $n^2<D<(n+1)^2$, $\sqrt{D}$, $D=2$, $n=1...","[elementary-number-theory, proof-writing, radi..."
926325,1027011,> A *countably infinite* set is a set for whic...,"[$a_1,a_2,a_3,...$, $0,1,-1,2,-2,3,-3,...$, $\...","[elementary-set-theory, intuition, infinity, faq]"


In [28]:
# смотрим уникальные формулы
unique_forms = extracted_phrases_dataframe['original_formula'].unique()

In [29]:
# количество достаточно длинных уникальных формул
i = 0
for form in unique_forms:
    if len(form)>50:
        i+=1
print(i)

1175201


In [34]:
# переводим в удобный формат
# сохраняем таблицу с формулами
extracted_phrases_dataframe.to_parquet("formulas_table_3-8.parquet")

In [33]:
# сохраняем таблицу с текстами для перевода
fin_df.to_parquet("markdown_table_3-8.parquet")